<a href="https://colab.research.google.com/github/tomasonjo/blogs/blob/master/llm/enhancing_rag_with_graph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install --upgrade --quiet  langchain langchain-community langchain-openai langchain-experimental neo4j wikipedia tiktoken yfiles_jupyter_graphs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.0/974.0 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.0/203.0 kB 16.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 43.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.7/314.7 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
import os
try:
  from google.colab import userdata
except:
  pass

os.environ["OPENAI_API_KEY"] = userdata.get('OXFORD_OPENAI_API_KEY')
os.environ["NEO4J_URI"] = userdata.get('NEO4J_URI')
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = userdata.get('NEO4J_PASSWORD')

In [11]:
# pip install --upgrade --quiet  langchain
# pip install --upgrade --quiet  langchain-openai

from langchain.chains import GraphCypherQAChain
from langchain_community.graphs import Neo4jGraph
from langchain_openai import ChatOpenAI

graph = Neo4jGraph(
    url=os.environ["NEO4J_URI"],
    username=os.environ["NEO4J_USERNAME"],
    password=os.environ["NEO4J_PASSWORD"]
)

def drop_nodes(graph):
  cypher_query = "MATCH (n) DETACH DELETE n"
  graph.query(cypher_query)
  print("Deleted all the existing records")


def load_data(graph):
  # Load Kellys Dataset
  graph.query(
  """
    LOAD CSV WITH HEADERS FROM 'https://docs.google.com/spreadsheets/d/1gxEEnp0NklCS7ywsU-IgM5LAY2X74aVoDH0R4-PdNyI/export?format=csv' AS row
    WITH row,
      apoc.date.parse(row.`Date of Sale`, 'ms', 'MM/dd/yyyy') AS parsedDate
    MERGE (c:Country {name: row.Country})
    MERGE (s:Store {id: toInteger(row.`Store ID`)})
    ON CREATE SET s.country = row.Country
    MERGE (pc:ProductCategory {name: row.`Product Category`})
    MERGE (p:Product {id: toInteger(row.`Product ID`)})
    ON CREATE SET p.category = row.`Product Category`
    MERGE (sale:Sale {id: toInteger(row.ID), unitsSold: toInteger(row.`Units Sold`), date: date(datetime({epochMillis: parsedDate})), price: toFloat(row.`Price Sold`), gdpGrowth: toFloat(row.`GDP Growth Rate`), inflation: toFloat(row.`Inflation Rate`)})
    MERGE (s)-[:LOCATED_IN]->(c)
    MERGE (s)-[:SOLD]->(sale)
    MERGE (sale)-[:OF_PRODUCT]->(p)
    MERGE (p)-[:BELONGS_TO]->(pc)
  """
  )
  print("Loaded Kellys dataset to Neo4J")

drop_nodes(graph)
load_data(graph)


ERROR:neo4j.io:Failed to write data to connection IPv4Address(('6f42d5f2.databases.neo4j.io', 7687)) (ResolvedIPv4Address(('34.126.64.110', 7687)))


Deleted all the existing records
Loaded Kellys dataset to Neo4J


In [12]:

chain = GraphCypherQAChain.from_llm(
    ChatOpenAI(model="gpt-4o", temperature=0), graph=graph, verbose=True
)

chain.run("Get all the products sold in store 8036")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (s:Store {id: 8036})-[:SOLD]->(sale:Sale)-[:OF_PRODUCT]->(p:Product)
RETURN p

Full Context:
[{'p': {'id': 18702, 'category': 'Mens'}}, {'p': {'id': 72532, 'category': 'Mens'}}, {'p': {'id': 39715, 'category': 'Mens'}}, {'p': {'id': 80432, 'category': 'Mens'}}, {'p': {'id': 60050, 'category': 'Mens'}}, {'p': {'id': 80512, 'category': 'Mens'}}, {'p': {'id': 71258, 'category': 'Womens'}}, {'p': {'id': 13386, 'category': 'Womens'}}, {'p': {'id': 65330, 'category': 'Womens'}}, {'p': {'id': 31672, 'category': 'Womens'}}]

> Finished chain.


"I don't know the answer."

In [13]:
output = chain.run("Get all the stores in UK country with relations in the result")
print(output)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (s:Store)-[r1:LOCATED_IN]->(c:Country {name: 'UK'})
OPTIONAL MATCH (s)-[r2:SOLD]->(sale:Sale)-[r3:OF_PRODUCT]->(p:Product)-[r4:BELONGS_TO]->(pc:ProductCategory)
RETURN s, r1, c, r2, sale, r3, p, r4, pc



ERROR:neo4j.io:Failed to write data to connection IPv4Address(('6f42d5f2.databases.neo4j.io', 7687)) (ResolvedIPv4Address(('34.126.64.110', 7687)))


Full Context:
[{'s': {'country': 'Spain', 'id': 8036}, 'r1': ({'country': 'Spain', 'id': 8036}, 'LOCATED_IN', {'name': 'UK'}), 'c': {'name': 'UK'}, 'r2': ({'country': 'Spain', 'id': 8036}, 'SOLD', {'date': neo4j.time.Date(2024, 1, 24), 'gdpGrowth': 7.07, 'price': 50.1, 'unitsSold': 2, 'id': 1, 'inflation': 0.17}), 'sale': {'date': neo4j.time.Date(2024, 1, 24), 'gdpGrowth': 7.07, 'price': 50.1, 'unitsSold': 2, 'id': 1, 'inflation': 0.17}, 'r3': ({'date': neo4j.time.Date(2024, 1, 24), 'gdpGrowth': 7.07, 'price': 50.1, 'unitsSold': 2, 'id': 1, 'inflation': 0.17}, 'OF_PRODUCT', {'id': 18702, 'category': 'Mens'}), 'p': {'id': 18702, 'category': 'Mens'}, 'r4': ({'id': 18702, 'category': 'Mens'}, 'BELONGS_TO', {'name': 'Mens'}), 'pc': {'name': 'Mens'}}, {'s': {'country': 'Spain', 'id': 8036}, 'r1': ({'country': 'Spain', 'id': 8036}, 'LOCATED_IN', {'name': 'UK'}), 'c': {'name': 'UK'}, 'r2': ({'country': 'Spain', 'id': 8036}, 'SOLD', {'date': neo4j.time.Date(2023, 11, 12), 'gdpGrowth': 6.58, 'p

In [ ]:
chain.run("Get me the top highest number of units sold in UK country")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (s:Store)-[:LOCATED_IN]->(c:Country {name: 'UK'})-[:SOLD]->(sale:Sale)
RETURN sale.unitsSold
ORDER BY sale.unitsSold DESC
LIMIT 1

Full Context:
[]

> Finished chain.


"I don't know the answer."